In [103]:
from google.cloud import bigquery
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import re



In [79]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer("https://www.kaggle.com/models/google/gnews-swivel/TensorFlow2/tf2-preview-20dim/1", output_shape=[],
                           input_shape=[], dtype=tf.string) 

In [96]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
valid = pd.read_csv('validation.csv')
train.head()

,text,label
0,This was an absolutely terrible movie. Don't b...,0
1,"I have been known to fall asleep during films,...",0
2,Mann photographs the Alberta Rocky Mountains i...,0
3,This is the kind of film for a snowy Sunday af...,1
4,"As others have mentioned, all the women that g...",1


In [120]:
train.dtypes


text            object
label            int64
preproc_text    object
dtype: object

In [101]:
def preprocess_text(text):
    text = text.lower()
    text = text.replace("<br />", " ")
    text = ''.join([c if c.isalpha() or c.isspace() else ' ' for c in text])
    text = text.strip()
    return text

train['preproc_text'] = train['text'].apply(preprocess_text)
test['preproc_text'] = test['text'].apply(preprocess_text)
train.head()

,text,label,preproc_text
0,This was an absolutely terrible movie. Don't b...,0,this was an absolutely terrible movie don t b...
1,"I have been known to fall asleep during films,...",0,i have been known to fall asleep during films ...
2,Mann photographs the Alberta Rocky Mountains i...,0,mann photographs the alberta rocky mountains i...
3,This is the kind of film for a snowy Sunday af...,1,this is the kind of film for a snowy sunday af...
4,"As others have mentioned, all the women that g...",1,as others have mentioned all the women that g...


In [176]:
train.preproc_text


0        this was an absolutely terrible movie  don t b...
1        i have been known to fall asleep during films ...
2        mann photographs the alberta rocky mountains i...
3        this is the kind of film for a snowy sunday af...
4        as others have mentioned  all the women that g...
                               ...                        
14995    i saw this movie in       i was    or     when...
14996    the only reason i give this movie an   out of ...
14997    i saw this movie in my childhood  and after   ...
14998    could contain spoilers     i m surprised by th...
14999    long  boring  blasphemous  never have i been s...
Name: preproc_text, Length: 15000, dtype: object

In [128]:
# Hyperparameters
vocab_size = 10000
max_length = 100

# Create the TextVectorization layer
vectorizer = TextVectorization(max_tokens=vocab_size, output_mode='int', output_sequence_length=max_length, pad_to_max_tokens=True)

# Adapt the vectorizer to the training data
vectorizer.adapt(train['preproc_text'])
vectorizer(train['preproc_text'].iloc[0])

<tf.Tensor: shape=(100,), dtype=int64, numpy=
array([  11,   14,   35,  443,  373,   17,   88,   22,   29, 9059,    9,
         34, 1288, 3495,   41,  498,    1,  198,   27,   84,  157,   19,
         11,  206,  329,   29,   68,  245,  218,    9,  470,   60,   68,
         84,  114,   97,   24, 6098,   11,   17,   13,  664,  758,   11,
         17,    7,   35,  402, 7601,  179, 2363,  414,    2,   90, 1148,
        136,   71,  147,   54,    2,    1, 7577,   71,  228,   68, 3022,
         16,    1, 2551,    1,    1, 1468, 4505,    3,   42, 3950,  116,
       1680,   18, 3495,   14,  161,   19,    4, 1148,  879, 7408,    9,
          4,   17,   12,   14, 3912,    5,  100,  146, 1115,   10,  230,
        665])>

In [129]:
sample_input = tf.constant([(train.iloc[0]['text'])])
hub_layer(sample_input)

<tf.Tensor: shape=(1, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ]],
      dtype=float32)>

In [169]:
#old
model = tf.keras.models.Sequential([
  hub_layer,
  keras.layers.Reshape((20, 1)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(40, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
  tf.keras.layers.Dense(1, activation='softmax')
])

In [ ]:
#old
model = tf.keras.models.Sequential([
  hub_layer,
  tf.keras.layers.Dense(1000, activation='relu'),
  tf.keras.layers.Dense(500, activation='relu'),
  tf.keras.layers.Dense(250, activation='relu'),
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(16, activation='relu'),
  tf.keras.layers.Dense(1)
])

In [196]:
#current
model = tf.keras.models.Sequential([
   # keras.layers.Reshape(None,),
    tf.keras.layers.InputLayer(input_shape=[1], dtype="string"),
    vectorizer,
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=16, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
    tf.keras.layers.Dense(6, activation='relu'),
    #tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_47"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_5 (TextV  (None, 100)              0         
 ectorization)                                                   
                                                                 
 embedding_16 (Embedding)    (None, 100, 16)           160000    
                                                                 
 bidirectional_55 (Bidirecti  (None, 100, 40)          5920      
 onal)                                                           
                                                                 
 bidirectional_56 (Bidirecti  (None, 40)               9760      
 onal)                                                           
                                                                 
 dense_73 (Dense)            (None, 6)                 246       
                                                     

In [163]:
#type(train['preproc_text'].iloc[0])
train["preproc_text"] = train.preproc_text.astype(str)

In [170]:
batch_size= 100
train_ds = tf.data.Dataset.from_tensor_slices((train["preproc_text"], train["label"]))
train_ds = train_ds.shuffle(buffer_size=len(train["preproc_text"])).batch(batch_size)


In [ ]:
model.compile(optimizer='adam', 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

model.fit(train_ds, epochs = 6)

Epoch 1/6
150/150 [==============================] - 28s 136ms/step - loss: 0.6129 - accuracy: 0.4993
Epoch 2/6
150/150 [==============================] - 21s 142ms/step - loss: 0.3635 - accuracy: 0.4993
Epoch 3/6
150/150 [==============================] - 20s 134ms/step - loss: 0.2509 - accuracy: 0.4993
Epoch 4/6
150/150 [==============================] - 20s 135ms/step - loss: 0.1968 - accuracy: 0.4993
Epoch 5/6
150/150 [==============================] - 22s 144ms/step - loss: 0.1504 - accuracy: 0.4993
Epoch 6/6
112/150 [=====================>........] - ETA: 5s - loss: 0.1092 - accuracy: 0.5000

In [84]:
test_loss, test_acc = model.evaluate(test['text'], test['label'])
print('\nTest accuracy: {}'.format(test_acc))


782/782 [==============================] - 11s 11ms/step - loss: 0.5766 - sparse_categorical_accuracy: 0.5000

Test accuracy: 0.5


In [85]:
model.predict(test['text'])

782/782 [==============================] - 12s 12ms/step


array([[0.99999994],
       [0.99999994],
       [0.99999994],
       ...,
       [0.99999994],
       [0.99999994],
       [0.99999994]], dtype=float32)

In [4]:
model.save('model_v1')

INFO:tensorflow:Assets written to: model_v1/assets


INFO:tensorflow:Assets written to: model_v1/assets


In [5]:
BUCKET = 'reddit_raw_data_0184598608709384596' # use your own bucket name here
!gsutil cp -r model_v1 gs://{BUCKET}/

Copying file://model_v1/keras_metadata.pb [Content-Type=application/octet-stream]...
Copying file://model_v1/saved_model.pb [Content-Type=application/octet-stream]...
Copying file://model_v1/fingerprint.pb [Content-Type=application/octet-stream]...
Copying file://model_v1/variables/variables.data-00000-of-00001 [Content-Type=application/octet-stream]...
- [4 files][  1.7 MiB/  1.7 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://model_v1/variables/variables.index [Content-Type=application/octet-stream]...
Copying file://model_v1/assets/tokens.txt [Content-Type=text/plain]...          
- [6 files][  1.8 MiB/  1.8 MiB]                                                
Operation completed over 6 objects/1.8 MiB.             

In [6]:
def to_csv(data, filename):
  """Takes TF Dataset and writes it to a (local) CSV file. Make sure the dataset is not too large!"""
  import csv

  data_list = [{ 'text': text.decode('utf-8'), 'label': label } for text, label in data.as_numpy_iterator()]
  filename = '{}.csv'.format(filename)

  with open(filename, 'w') as f:
    writer = csv.DictWriter(f, data_list[0].keys())
    writer.writeheader()
    writer.writerows(data_list)

train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

to_csv(train_data, 'train')
to_csv(validation_data, 'validation')
to_csv(test_data, 'test')

BUCKET = 'reddit_raw_data_0184598608709384596' # use your own bucket name here
!gsutil cp train.csv gs://{BUCKET}/
!gsutil cp validation.csv gs://{BUCKET}/
!gsutil cp test.csv gs://{BUCKET}/

Copying file://train.csv [Content-Type=text/csv]...
/ [1 files][ 19.0 MiB/ 19.0 MiB]                                                
Operation completed over 1 objects/19.0 MiB.                                     
Copying file://validation.csv [Content-Type=text/csv]...
/ [1 files][ 12.8 MiB/ 12.8 MiB]                                                
Operation completed over 1 objects/12.8 MiB.                                     
Copying file://test.csv [Content-Type=text/csv]...
- [1 files][ 31.1 MiB/ 31.1 MiB]                                                
Operation completed over 1 objects/31.1 MiB.                                     


,text,label
0,This was an absolutely terrible movie. Don't b...,0
1,"I have been known to fall asleep during films,...",0
2,Mann photographs the Alberta Rocky Mountains i...,0
3,This is the kind of film for a snowy Sunday af...,1
4,"As others have mentioned, all the women that g...",1


ValueError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 1025, in train_step
        self._validate_target_and_loss(y, loss)
    File "/opt/conda/lib/python3.10/site-packages/keras/engine/training.py", line 989, in _validate_target_and_loss
        raise ValueError(

    ValueError: No loss found. You may have forgotten to provide a `loss` argument in the `compile()` method.
